In [61]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('weatherAUS.csv', sep = ',', header = 0)
display(df.head(1))

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No


In [62]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://image.jimcdn.com/app/cms/image/transf/dimension=640x1024:format=jpg/path/s61405ffb1b1ac796/image/if04a015502610c01/version/1506947682/image.jpg")


N = 1 NNE = 2 NE = 3 ENE = 4 E = 5 ESE = 6 SE = 7 SSE = 8 S = 9 SSW = 10 SW = 11 WSW = 12 W = 13 WNW = 14 NW = 15 NNW = 16

0 = Sans vent ni direction

In [63]:
isna_9am = 0
isna_3pm = 0

def directions_to_num():
    dict = {'N': 1, 'NNE': 2, 'NE': 3, 'ENE': 4, 'E': 5, 'ESE': 6, 'SE': 7, 'SSE': 8, 'S': 9, 'SSW': 10, 'SW': 11, 'WSW': 12, 'W': 13, 'WNW': 14, 'NW': 15, 'NNW': 16}
    df[['WindDir9am', 'WindDir3pm', 'WindGustDir']] = df[['WindDir9am', 'WindDir3pm', 'WindGustDir']].replace(dict)


def convert (number):
    
    """ This fonction convert number values to degrees."""
        
    match number:
        case 1:
             deg = 0.0
        case 2:
             deg = 22.5
        case 3:
             deg = 45.0
        case 4:
             deg = 67.5        
        case 5:
             deg = 90.0
        case 6:
             deg = 112.5
        case 7:
             deg = 135.0
        case 8:
             deg = 157.5
        case 9:
             deg = 180.0        
        case 10:
             deg = 202.5
        case 11:
             deg = 225.0
        case 12:
             deg = 247.5
        case 13:
             deg = 270.0
        case 14:
             deg = 292.5        
        case 15:
             deg = 315.0
        case 16:
             deg = 337.5    
    
    return deg
            
def convert2 (number):
    
    """This fonction convert degrees to nearest right hand (clockwise) number value. """
    
    match number:
        case num if num == 0:
             deg = 1
        case num if 0 < num <=  22.5:
             deg = 2
        case num if 22.5 < num <= 45.0:
             deg = 3
        case num if 45.0 < num <= 67.5:
             deg = 4
        case num if 67.5 < num <= 90.0:
             deg = 5        
        case num if 90.0 < num <= 112.5:
             deg = 6
        case num if 112.5 < num <= 135.0:
             deg = 7
        case num if 135.0 < num <= 157.5:
             deg = 8
        case num if 157.5 < num <= 180.0:
             deg = 9
        case num if 180.0 < num <= 202.5:
             deg = 10        
        case num if 202.5 < num <= 225.0:
             deg = 11
        case num if 225.0 < num <= 247.5:
             deg = 12
        case num if 247.5 < num <= 270.0:
             deg = 13
        case num if 270.0 < num <= 292.5:
             deg = 14
        case num if 292.5 < num <= 315.0:
             deg = 15        
        case num if 315.0 < num <= 337.5:
             deg = 16
        case num if 337.5 < num <= 360.0:
             deg = 1    
    
    return deg

def calculate_middle(before, after):
    
    """This fonction finds the shortest middle between two degrees."""
    
    before = convert(before)
    after = convert(after)
    
    # If before is bigger than after
    # we exchange the values
    # in order to have the absolute difference
    if before > after:
        temp = before
        before = after
        after = temp
    
    # If the absolute difference is larger than
    # 180 we deduct 360 from after (now the 
    # the largest of the two) in order to get the
    # shortest distance.
    if after - before > 180: 
        after -= 360
    
    # After is now the distance to 0 and we this to 
    # befor which is the distance from 0 to before,
    # that is the shortest distance between the two
    # we take half of that.
    final = (after + before)/2
 
    # If the result is negative we add 360 to get the
    # degrees that mark halfway.
    if final < 0:
        final += 360
    
    # We convert to nearest degrre clockwisew
    final = convert2(final)
    
    return final

def replace_middle_9am(df):
    
    end = len(df) - 1
    
    # For each line in df except the last to avoid index out of bound
    
    for i in range (0, end):
        
        # i should not be 0 to avoid index out of bound when reading the precedent line
        # and the location should be identical to location in precedent line to avoid
        # shift between cities
        
        if i != 0 and df.loc[i, 'Location'] == df.loc[i-1, 'Location']:
            
            # There should be a Nan in this line and windspeed not 0.
            # Zero windspeed is delt with in another function
            
            if pd.isna(df.loc[i, 'WindDir9am']) and df.loc[i, 'WindSpeed9am'] != 0:
                
                # No NaN at 3 pm and no NaN at 3pm the day before and neither is 0,
                # that is no direction
                
                if pd.notna(df.loc[i, 'WindDir3pm']) and pd.notna(df.loc[i-1, 'WindDir3pm']) and df.loc[i, 'WindDir3pm'] != 0 and df.loc[i-1, 'WindDir3pm'] != 0:
                    
                    #print(df.loc[i, 'WindDir3pm'], df.loc[i-1, 'WindDir3pm'])
                    
                    # Calculate middle between direction at 3pm the day before and 3pm the
                    # same day.
                    
                    res = calculate_middle(df.loc[i, 'WindDir3pm'], df.loc[i-1, 'WindDir3pm'])
                    
                    # Replave NaN with the middle value
                    
                    df.at[i, 'WindDir9am'] = res
                    
                    #print(df.loc[i, 'WindDir9am'])    
    
    isn = df['WindDir9am'].isna().sum()
    print("Number of NaN's in column 'WindDir9am' after 'replace middle': ", isn)
                    
    return df

def replace_middle_3pm(df):
    
    end = len(df) - 1
    
    # For each line in df except the last to avoid index out of bound
    
    for i in range (0, end):
        
        # i should not be 0 to avoid index out of bound when reading the precedent line
        # and the location should be identical to location in precedent line to avoid
        # shift between cities
        
        # Anyway the first and the last record cannot have a middle
        
        if i != 0 and df.loc[i, 'Location'] == df.loc[i+1, 'Location']:
            
            # There should be a Nan in this line and windspeed not 0.
            # Zero windspeed is delt with in another function
            
            if pd.isna(df.loc[i, 'WindDir3pm']) and df.loc[i, 'WindSpeed3pm'] != 0:
                if pd.notna(df.loc[i, 'WindDir9am']) and pd.notna(df.loc[i+1, 'WindDir9am']) and df.loc[i, 'WindDir9am'] != 0 and df.loc[i+1, 'WindDir9am'] != 0:
                    
                    #print(df.loc[i, 'WindDir9am'], df.loc[i+1, 'WindDir9am'])
                    
                    # Calculate middle between direction at 3pm the day before and 3pm the
                    # same day.
                    
                    res = calculate_middle(df.loc[i, 'WindDir9am'], df.loc[i+1, 'WindDir9am'])
                    
                    # Replave NaN with the middle value
                    
                    df.at[i, 'WindDir3pm'] = res
                    
                    #print(df.loc[i, 'WindDir3pm'])
                    
    isn = df['WindDir3pm'].isna().sum()
    print("Number of NaN's in column 'WindDir3pm' after 'replace middle': ", isn)

    return df

def replace_middle_gust(df):
    
    end = len(df) - 1
    
    # For each line in df except the last to avoid index out of bound
    
    for i in range (0, end):
        
        # i should not be 0 to avoid index out of bound when reading the precedent line
        # and the location should be identical to location in precedent line to avoid
        # shift between cities
        
        # Anyway the first and the last record cannot have a middle
        
        if i != 0 and df.loc[i, 'Location'] == df.loc[i+1, 'Location'] and df.loc[i, 'Location'] == df.loc[i-1, 'Location']:
            
            # There should be a Nan in this line 
            
            if pd.isna(df.loc[i, 'WindGustDir']):
                if pd.notna(df.loc[i-1, 'WindGustDir']) and pd.notna(df.loc[i+1, 'WindGustDir']) and df.loc[i-1, 'WindGustDir'] != 0 and df.loc[i+1, 'WindGustDir'] != 0:
                    
                    #print(df.loc[i, 'WindGustDir'], df.loc[i+1, 'WindGustDir'])
                    
                    # Calculate middle between direction at 3pm the day before and 3pm the
                    # same day.
                    
                    res = calculate_middle(df.loc[i-1, 'WindGustDir'], df.loc[i+1, 'WindGustDir'])
                    
                    # Replave NaN with the middle value
                    
                    df.at[i, 'WindGustDir'] = res
                    
                    #print(df.loc[i, 'WindGustDir'])
                    
    isn = df['WindGustDir'].isna().sum()
    print("Number of NaN's in column 'WindGustDir' after 'replace middle': ", isn)

    return df

def replace_nowind_9am (df):
    
    end = len(df)
    
    for i in range (0, end):
        
        # If direction is NaN and wind speed is 0
        
        if pd.isna(df.loc[i, 'WindDir9am']) and df.loc[i, 'WindSpeed9am'] == 0:
            
            # Replace wind direction with 0
            
            df.at[i, 'WindDir9am'] = 0
            
            #print(df.loc[i, 'WindDir9am'])
    
    isn = df['WindDir9am'].isna().sum()
    print("Number of NaN's in column 'WindDir9am' after 'replace nowind': ", isn)
    
    return(df)
    
def replace_nowind_3pm (df):
    
    end = len(df)
    
      
    for i in range (0, end):
        
        # If direction is NaN and wind speed is 0
            
        if pd.isna(df.loc[i, 'WindDir3pm']) and df.loc[i, 'WindSpeed3pm'] == 0:
            
            # Replace wind direction with 0
            
            df.at[i, 'WindDir3pm'] = 0
            
            #print(df.loc[i, 'WindDir3pm'])

    isn = df['WindDir3pm'].isna().sum()
    print("Number of NaN's in column 'WindDir3pm' after 'replace nowind': ", isn)
    
    return df

def replace_sameday_9am (df):
    
    end = len(df)
    
    for i in range (0, end):
            
        # If direction at 9am is NaN and direction at 3pm is not
        
        if pd.isna(df.loc[i, 'WindDir9am']) and pd.notna(df.loc[i, 'WindDir3pm']):
                
                # Set wind direction at 9am equal to the wind direction at 3pm (same day)
                
                df.at[i, 'WindDir9am'] = df.loc[i, 'WindDir3pm']
                
                #print(df.loc[i, 'WindDir9am'])
    
    isn = df['WindDir9am'].isna().sum()
    print("Number of NaN's in column 'WindDir9am' after 'replace sameday': ", isn)
    
    return df

def replace_sameday_3pm (df):
    
    end = len(df)
    
    for i in range (0, end):
        
        # If direction at 3pm is NaN and direction at 9am is not
        
        if pd.isna(df.loc[i, 'WindDir3pm']) and pd.notna(df.loc[i, 'WindDir9am']):
            
            # Set wind direction at 3pm equal to the wind direction at 9am (same day)
            
            df.at[i, 'WindDir3pm'] = df.loc[i, 'WindDir9am']
            
            #print(df.loc[i, 'WindDir3pm'])
    
    isn = df['WindDir3pm'].isna().sum()
    print("Number of NaN's in column 'WindDir3pm' after 'replace sameday': ", isn)
    
    return df

def replace_mode_9am (df):
    
    end = len(df)
    
    # Mode for the direction in the same city and hour. If double mode, the first is chosen.
    
    mod_9am = df.groupby(df['Location'])['WindDir9am'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()
    mod_9am = mod_9am.reset_index(level=['Location'])
    
    for i in range (0, end):
        
        # If direction is NaN replacew with mode.
        
        if pd.isna(df.loc[i, 'WindDir9am']):                
            df.at[i, 'WindDir9am'] = mod_9am[mod_9am['Location'] == df.loc[i, 'Location']]['WindDir9am']
            
            #print("9am", df.loc[i, 'WindDir9am'])
    
    isn = df['WindDir9am'].isna().sum()
    print("Number of NaN's in column 'WindDir9am' after 'replace mode': ", isn)

    return df

def replace_mode_3pm (df):
    
    end = len(df)
    
    # Mode for the direction in the same city and hour. If double mode, the first is chosen.
    
    mod_3pm = df.groupby(df['Location'])['WindDir3pm'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()
    mod_3pm = mod_3pm.reset_index(level=['Location'])
    
    for i in range (0, end):
        
        # If direction is NaN replacew with mode.
    
        if pd.isna(df.loc[i, 'WindDir3pm']):
        
            df.at[i, 'WindDir3pm'] = mod_3pm[mod_3pm['Location'] == df.loc[i, 'Location']]['WindDir3pm']
        
            #print("3pm", df.loc[i, 'WindDir3pm'])
    
    isn = df['WindDir3pm'].isna().sum()
    print("Number of NaN's in column 'WindDir3pm' after 'replace mode': ", isn)
    
    return df
    
def replace_mode_gust (df):
    
    end = len(df)
    
    # Mode for the direction in the same city and hour. Si double mode, le premier est choisi.
     
    mod_gust = df.groupby(df['Location'])['WindGustDir'].agg(lambda x: pd.Series.mode(x, dropna=False)[0]).to_frame()
    mod_gust = mod_gust.reset_index(level=['Location'])

    #print(mod_gust)

    for i in range (0, end):

        # If direction is NaN replace with mode.

        if pd.isna(df.loc[i, 'WindGustDir']):

            df.at[i, 'WindGustDir'] = mod_gust[mod_gust['Location'] == df.loc[i, 'Location']]['WindGustDir']

            #print("GustDir", df.loc[i, 'WindGustDir'])

    isn = df['WindGustDir'].isna().sum()
    print("Number of NaN's in column 'WindGustDir' after 'replace mode': ", isn)
    
    
    return df

def replace_clust_gust(df):
    
    end = len(df)
    
    dict = {'Albury': [4], 'BadgerysCreek': [6], 'Cobar': [4], 'CoffsHarbour': [2], 'Moree': [4],
       'Newcastle': [5], 'NorahHead': [2], 'NorfolkIsland': [2], 'Penrith': [6], 'Richmond': [6],
       'Sydney': [5], 'SydneyAirport': [5], 'WaggaWagga': [4], 'Williamtown': [5],
       'Wollongong': [5], 'Canberra': [7], 'Tuggeranong': [7], 'MountGinini': [8], 'Ballarat': [7],
       'Bendigo': [6], 'Sale': [6], 'MelbourneAirport': [6], 'Melbourne': [6], 'Mildura': [4],
       'Nhil': [4], 'Portland': [6], 'Watsonia': [6], 'Dartmoor': [6], 'Brisbane': [3], 'Cairns': [1],
       'GoldCoast': [2], 'Townsville': [1], 'Adelaide': [5], 'MountGambier': [6], 'Nuriootpa': [6],
       'Woomera': [4], 'Albany': [6], 'Witchcliffe': [5], 'PearceRAAF': [5], 'PerthAirport': [5],
       'Perth': [5], 'SalmonGums': [4], 'Walpole': [6], 'Hobart': [7], 'Launceston': [7],
       'AliceSprings': [3], 'Darwin': [1], 'Katherine': [1], 'Uluru': [3]}
       
    df['Climate_Type'] = df['Location']
    
    df['Climate_Type'] = df['Climate_Type'].replace(dict)
    
    
    #display(df.head())
    
    # Mode for the direction in the same climate zone. If double mode, the first is chosen.
    
    mod_gust = df.groupby(df['Climate_Type'])['WindGustDir'].agg(lambda x: pd.Series.mode(x, dropna=True)[0]).to_frame()
    mod_gust = mod_gust.reset_index(level=['Climate_Type'])
    
    #display(mod_gust)
    
    for i in range (0, end):
        
        # If direction is NaN replacew with mode for climate type.
        
        if pd.isna(df.loc[i, 'WindGustDir']):                
            df.at[i, 'WindGustDir'] = mod_gust[mod_gust['Climate_Type'] == df.loc[i, 'Climate_Type']]['WindGustDir']
    
    isn = df['WindGustDir'].isna().sum()
    print("Number of NaN's in column 'WindGustDir' after 'replace clust': ", isn)
    
    df = df.drop('Climate_Type', axis=1)
    
    return df





In [64]:


def fill_nan_dir9am(df):
    df = replace_middle_9am(df)
    df = replace_nowind_9am(df)
    df = replace_sameday_9am(df)
    df = replace_mode_9am(df)
    
    return df

def fill_nan_dir3pm(df):
    df = replace_middle_3pm(df)
    df = replace_nowind_3pm(df)
    df = replace_sameday_3pm(df)
    df = replace_mode_3pm(df)
    
    return df

def fill_nan_dirGust(df):
    df = replace_middle_gust(df)
    df = replace_mode_gust(df)
    df = replace_clust_gust(df)
    
    return df

In [65]:
directions_to_num()
df = fill_nan_dir9am(df)
df = fill_nan_dir3pm(df)
df = fill_nan_dirGust(df)


df.head()

df.to_csv('df_cat_clean.csv')

Number of NaN's in column 'WindDir9am' after 'replace middle':  10163
Number of NaN's in column 'WindDir9am' after 'replace nowind':  1418
Number of NaN's in column 'WindDir9am' after 'replace sameday':  1276


C:\Users\chrom\AppData\Local\Temp\ipykernel_29800\263287831.py:349: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[i, 'WindDir9am'] = mod_9am[mod_9am['Location'] == df.loc[i, 'Location']]['WindDir9am']


Number of NaN's in column 'WindDir9am' after 'replace mode':  0
Number of NaN's in column 'WindDir3pm' after 'replace middle':  1990
Number of NaN's in column 'WindDir3pm' after 'replace nowind':  878
Number of NaN's in column 'WindDir3pm' after 'replace sameday':  0
Number of NaN's in column 'WindDir3pm' after 'replace mode':  0
Number of NaN's in column 'WindGustDir' after 'replace middle':  8967


C:\Users\chrom\AppData\Local\Temp\ipykernel_29800\263287831.py:399: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[i, 'WindGustDir'] = mod_gust[mod_gust['Location'] == df.loc[i, 'Location']]['WindGustDir']


Number of NaN's in column 'WindGustDir' after 'replace mode':  7078


C:\Users\chrom\AppData\Local\Temp\ipykernel_29800\263287831.py:443: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[i, 'WindGustDir'] = mod_gust[mod_gust['Climate_Type'] == df.loc[i, 'Climate_Type']]['WindGustDir']


Number of NaN's in column 'WindGustDir' after 'replace clust':  0


https://www.yourhome.gov.au/getting-started/australian-climate-zones

https://ncc.abcb.gov.au/sites/default/files/resources/2020//ClimateZoneMapAUST.pdf

Zone 1 Hot humid summer, warm winter

Zone 2 Warm humid summer, mild winter

Zone 3 Hot dry summer, warm winter

Zone 4 Hot dry summer, cool winter

Zone 5 Warm temperate

Zone 6 Mild temperate

Zone 7 Cool temperate

zone 8 Alpine


